In [1]:
#!pip install xgboost

In [2]:
import pandas as pd
import numpy as np
import seaborn as sb
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb

# 使用するモデルのインポート
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings('ignore')


train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
PassengerId = test['PassengerId']
full_data = [train, test]

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# modify data、add new features
# add "Has Cabin" feature which mean passnger has own cabin or not
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

# 同乗した兄弟、配偶者の人数（SibSp）と同乗した親、子供の人数（Parch）からファミリーサイズを追加する
# add "FamliSize" feature caliculated from SibSp and Parch.
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
# ファミリーサイズから1名の乗客かどうかを追加する
# add "IsAlone" from "FamiliSize"
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

# 乗船した港（Embarked）に欠損値があるため、Southamptonで補完する
# There are lack of data in "Embarked" column so complement data as "S".
# C = Cherbourg, Q = Queenstown, S = Southampton
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')

# 運賃（Fare）に欠損値があるため、中央値で補完する
#"There are lack of data in "Fare" column so complement data with median.
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())

# 年齢（Age）に欠損値があるため、平均と標準偏差から乱数を生成し、補完する
#"There are lack of data in "Age" column so complement data with ramdam value from mean and standard deviation.
for dataset in full_data:
    # 年齢の平均
    age_avg = dataset['Age'].mean()
    # 年齢の標準偏差
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)

# 乗客の名前から敬称（MrやMsなど）を抽出してくる関数を定義する
#def get_title(name):
#    title_search = re.search(' ([A-Za-z]+)\.', name)
#    if title_search:
#        return title_search.group(1)
#    return ""

# 乗客の名前（Name）から敬称を抽出し、追加する
#for dataset in full_data:
#    dataset['Title'] = dataset['Name'].apply(get_title)
# 敬称（Title）をグルーピングしたもので書き換える
#for dataset in full_data:
#    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

#    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
#    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
#    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

# データの整形
for dataset in full_data:
    # 性別（Sex）を定数化する
    #Categorize "Sex"
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

    # 敬称（Title）を定数化する
    #Categorize 
    #title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    #dataset['Title'] = dataset['Title'].map(title_mapping)
    # 敬称（Title）がないレコードを0埋めする
    #dataset['Title'] = dataset['Title'].fillna(0)

    # 乗船した港（Embarked）を定数化する
    #Categorize "Embarked"
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

    # 運賃（Fare）を定数化する
    #Categorize "Fare"
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare']                               = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare']                                  = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

    # 年齢（Age）を定数化する
    #Categorize "Age"
    dataset.loc[ dataset['Age'] <= 16, 'Age']                          = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4 ;

# 特徴量の選択（不要なカラムを落とす）
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
train = train.drop(drop_elements, axis = 1)
test  = test.drop(drop_elements, axis = 1)
train.head()

,Survived,Pclass,Sex,Age,Parch,Fare,Embarked,Has_Cabin,FamilySize,IsAlone
0,0,3,1,1,0,0,0,0,2,0
1,1,1,0,2,0,3,1,1,2,0
2,1,3,0,1,0,1,0,0,1,1
3,1,1,0,2,0,3,0,1,2,0
4,0,3,1,2,0,1,0,0,1,1


In [5]:
ntrain = train.shape[0]
ntest = test.shape[0]

# 交差検証の回数
NFOLDS = 5
# 同じ乱数を発生させるための固定値
SEED = 0
# パラメータの検証、学習モデルの精度を評価
# 交差検証（クロスバリデーション）でパラメータを検証し、精度がよく、過学習が起こらないパラメータを決定する
kf = KFold(n_splits=NFOLDS, random_state=SEED)

# Sklearn classifier を拡張
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)

    def fit(self,x,y):
        return self.clf.fit(x,y)

def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [6]:
# モデルのパラメータ定義
# Random Forest のパラメータ
rf_params = { 'n_jobs': -1, 'n_estimators': 500, 'warm_start': True, 'max_depth': 6, 'min_samples_leaf': 2, 'max_features' : 'sqrt', 'verbose': 0 }
# Extra Trees のパラメータ
et_params = { 'n_jobs': -1, 'n_estimators':500, 'max_depth': 8, 'min_samples_leaf': 2, 'verbose': 0 }
# AdaBoost のパラメータ
ada_params = { 'n_estimators': 500, 'learning_rate' : 0.75 }
# Gradient Boosting のパラメータ
gb_params = { 'n_estimators': 500, 'max_depth': 5, 'min_samples_leaf': 2, 'verbose': 0 }
# Support Vector Classifier のパラメータ 
svc_params = { 'kernel' : 'linear', 'C' : 0.025 }

# モデルのオブジェクト生成
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
svc = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)

# 学習データの生存（Survived）データ、学習データ、テストデータで配列を作成する
#print(train.head())
y_train = train['Survived'].ravel()
train = train.drop(['Survived'], axis=1)
x_train = train.values
x_test = test.values

# 第1段階の学習と予測を実行する
et_oof_train, et_oof_test = get_oof(et, x_train, y_train, x_test) # Extra Trees Classifier
rf_oof_train, rf_oof_test = get_oof(rf,x_train, y_train, x_test) # Random Forest Classifier
ada_oof_train, ada_oof_test = get_oof(ada, x_train, y_train, x_test) # AdaBoost Classifier
gb_oof_train, gb_oof_test = get_oof(gb,x_train, y_train, x_test) # Gradient Boost Classifier
svc_oof_train, svc_oof_test = get_oof(svc,x_train, y_train, x_test) # Support Vector Classifier

In [7]:
#!pip install scikit-optimize

In [16]:
spaces = [
    (1500, 3000), # n_estimators,
    (3, 10), # max_depth,
    (0.5, 1), #subsamples
    (0.5, 1), #colsample_bytree
    (0.01, 0.2)#learning rate
    
    # ['linear', 'poly', 'rbf']
]
def f(x):
    clf = xgb.XGBClassifier(n_estimators=x[0], max_depth=x[1],subsample=x[2],colsample_bytree=x[3], learning_rate=x[4])
    clf.fit(x_train, y_train)
    return -1 * clf.score(x_train, y_train)

In [19]:
from skopt import gp_minimize
# 第1段階の予測結果をインプット
x_train = np.concatenate(( et_oof_train, rf_oof_train, ada_oof_train, gb_oof_train, svc_oof_train), axis=1)
x_test = np.concatenate(( et_oof_test, rf_oof_test, ada_oof_test, gb_oof_test, svc_oof_test), axis=1)
# 第2段階の学習と予測を実行する
#gbm = xgb.XGBClassifier(n_estimators= 2000, max_depth= 4, min_child_weight= 2, gamma=0.9, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic', nthread= -1, scale_pos_weight=1).fit(x_train, y_train)
gbm = gp_minimize(
    f, spaces,
    acq_func="EI",
    verbose=True,
    n_jobs=2,
    n_calls=100)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.8090
Function value obtained: -0.8373
Current minimum: -0.8373
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1.2845
Function value obtained: -0.8395
Current minimum: -0.8395
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 1.3314
Function value obtained: -0.8406
Current minimum: -0.8406
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.9365
Function value obtained: -0.8395
Current minimum: -0.8406
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 1.0362
Function value obtained: -0.8406
Current minimum: -0.8406
Iteration No: 6 started. 

Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 1.6725
Function value obtained: -0.8406
Current minimum: -0.8406
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 1.8667
Function value obtained: -0.8395
Current minimum: -0.8406
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 0.8258
Function value obtained: -0.8373
Current minimum: -0.8406
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 0.7877
Function value obtained: -0.8395
Current minimum: -0.8406
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 0.9935
Function value obtained: -0.8406
Current minimum: -0.8406
Iteration No: 46 st

Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 1.0931
Function value obtained: -0.8406
Current minimum: -0.8406
Iteration No: 82 started. Searching for the next optimal point.
Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 1.7696
Function value obtained: -0.8406
Current minimum: -0.8406
Iteration No: 83 started. Searching for the next optimal point.
Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 1.4994
Function value obtained: -0.8406
Current minimum: -0.8406
Iteration No: 84 started. Searching for the next optimal point.
Iteration No: 84 ended. Search finished for the next optimal point.
Time taken: 1.3583
Function value obtained: -0.8395
Current minimum: -0.8406
Iteration No: 85 started. Searching for the next optimal point.
Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 1.6083
Function value obtained: -0.8373
Current minimum: -0.8406
Iteration No: 86 st

In [20]:
gbm.x

[2431, 5, 0.7215394468916365, 0.8965088199534419, 0.1596762034165154]

In [21]:
best_x = gbm.x
#n_estimators=x[0], max_depth=x[1],subsample=x[2],colsample_bytree=x[3], learning_rate=x[4]
clf = xgb.XGBClassifier(n_estimators= best_x[0], max_depth= best_x[1], min_child_weight= 2, gamma=0.9, subsample=best_x[2], colsample_bytree=best_x[3], objective= 'binary:logistic', nthread= -1, scale_pos_weight=1,learning_rate=best_x[4]).fit(x_train, y_train)
predictions = clf.predict(x_test)


# 予測結果をCSV出力
StackingSubmission = pd.DataFrame({ 'PassengerId': PassengerId,'Survived': predictions })
StackingSubmission.to_csv("StackingSubmission.csv", index=False)